Output implies that Citations are lower (and Warnings higher) than expected for Asian, Other, and White drivers. Conversely, Citations are higher for Black drivers, and highest for Hispanic drivers.

# TO DO

* Do crosstab for all stops, not just fatalities/accidents/injuries
* Get PERCENTAGES so we can see magnitude

# Import & Setup

In [1]:
import sys
sys.path.append(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/My-Package")

import pandas as pd
import numpy as np
from myfunctions import clean_path

In [2]:

from scipy.stats import chi2_contingency

In [3]:

from IPython.display import display, HTML

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [5]:
from typing import Union

In [6]:

df = pd.read_csv(filepath_or_buffer=clean_path(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/Policing Thesis/Modified Dataset - 2021.csv")).set_index('Stop ID')

C:\Users\mikha\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (18,19,20,21,22,23,32,44,45,46,47,60) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Create bool citation/warning col with ESERO & SERO as nan values

In [7]:
l_ = []

for x in df['Violation Type']:
    if x in ('Citation', 'Warning'):
        l_.append(x)
    else:
        l_.append(np.nan)

df['Citation/Warning'] = l_

del l_

In [8]:
df['Fatal - Sparse Filled'].sum()

434.0

In [9]:
accident_df = df[df['Accident - Sparse Filled']==1]
accident_df.name='Accident DF'

fatality_df = df[df['Fatal - Sparse Filled']==1]
fatality_df.name='Fatality DF'

personal_injury_df = df[df['Personal Injury - Sparse Filled']==1]
personal_injury_df.name='Personal Injury DF'


# Chi Square

## Chi-Square Function & Output

In [10]:
def chi_square_contingency_test(dataframe:pd.DataFrame, display_tables=True) -> Union[pd.DataFrame, pd.DataFrame]:
    """Returns Actual, Expected contingency tables"""
    
    contingency_table = pd.crosstab([dataframe['Race']], [dataframe['Citation/Warning']])

    chi2_result = chi2_contingency(contingency_table)

    if display_tables:
        print(f"Test statistic == {chi2_result[0]}")
        print(f"P-Value == {'{:.40f}'.format(chi2_result[1])}")
        print(f"Degrees of Freedom == {chi2_result[2]}")


        print("Contingency table == ")
        display(contingency_table)

    expected_frequencies = pd.DataFrame(chi2_result[3])
    expected_frequencies.columns = contingency_table.columns
    expected_frequencies.index = contingency_table.index
    
    pct_difference_table = pd.DataFrame(((contingency_table-chi2_result[3]).div(contingency_table)*100).round(4))
    try:
        pct_difference_table.name = dataframe.name
    except:
        raise ValueError('Dataframe has no name')
    if display_tables:
        print(f"Expected Frequencies == ")
        display(expected_frequencies.round(0))
        
        print(f"Difference between Actual and Expected Frequencies ({dataframe.name}) (Actual - Expected) == \n (Negative => Less than expected")
        display(contingency_table-chi2_result[3].round(0))
        
        
        # DON'T THINK THIS DIVISION IS CORRECT. SHOULDN'T IT BE EQUAL (FLIPPED SIGN) ACROSS ROWS????
        ## nvm - more citations than warnings, so (e.g.) a % increase in citations means a large % decrease in warnings
        print(f"PERCENT Difference between Actual and Expected Frequencies ({dataframe.name}) (Actual - Expected) == \n (Negative => Less than expected")
        display(pct_difference_table)
    
    # return contingency_table, expected_frequencies
    return pct_difference_table

In [11]:
def create_multi_metric_table(dataframes:list, display_tables=False) -> pd.DataFrame:
    pct_differences_list = []
    for dataframe in dataframes:
        pct_difference_table = chi_square_contingency_test(dataframe=dataframe, display_tables=display_tables)
        pct_difference_table.drop(labels=[col for col in pct_difference_table if col!='Citation'], 
                                  axis='columns',
                                  inplace=True)
        
        source_label = pct_difference_table.name.split()[0]
        pct_difference_table.rename(columns={'Citation':source_label}, 
                                    inplace=True)
        
        # display(pct_difference_table)
        
        pct_differences_list.append(pct_difference_table)
        
    pct_differences = pd.concat(pct_differences_list, axis='columns')
    
    return pct_differences

In [16]:
multi_metric_freq_dif_table = create_multi_metric_table(dataframes=[accident_df, personal_injury_df, fatality_df])

In [19]:
multi_metric_freq_dif_table.name = None

In [22]:
multi_metric_freq_dif_table

Citation/Warning,Accident,Personal,Fatality
Race,,,
ASIAN,-3.8747,-2.6117,23.4295
BLACK,-0.1740,0.5068,11.1466
HISPANIC,3.4882,1.7331,2.4837
NATIVE AMERICAN,-3.5604,4.8494,-inf
OTHER,-4.4441,-4.1285,-60.7981
WHITE,-1.6797,-0.7873,-13.6805


In [29]:
N_df = pd.DataFrame({'Accident':accident_df['Race'].value_counts().sort_index(),
                    'Personal Injury':personal_injury_df['Race'].value_counts().sort_index(),
                    'Fatality':fatality_df['Race'].value_counts().sort_index()})

N_df.index.name="Race"
N_df

,Accident,Personal Injury,Fatality
Race,,,
ASIAN,3006,1172,25
BLACK,14750,5369,137
HISPANIC,15967,5647,96
NATIVE AMERICAN,116,42,1
OTHER,2766,1028,20
WHITE,19316,7479,155


## Accidents

In [35]:
chi_square_contingency_test(accident_df)

Test statistic == 350.62488701081014
P-Value == 0.0000000000000000000000000000000000000000
Degrees of Freedom == 5
Contingency table == 


Citation/Warning,Citation,Warning
Race,,
ASIAN,2607,374
BLACK,13298,1366
HISPANIC,14982,935
NATIVE AMERICAN,100,14
OTHER,2391,358
WHITE,17184,2050


Expected Frequencies == 


Citation/Warning,Citation,Warning
Race,,
ASIAN,2708.0,273.0
BLACK,13321.0,1343.0
HISPANIC,14459.0,1458.0
NATIVE AMERICAN,104.0,10.0
OTHER,2497.0,252.0
WHITE,17473.0,1761.0


Difference between Actual and Expected Frequencies (Accident DF) (Actual - Expected) == 
 (Negative => Less than expected


Citation/Warning,Citation,Warning
Race,,
ASIAN,-101.0,101.0
BLACK,-23.0,23.0
HISPANIC,523.0,-523.0
NATIVE AMERICAN,-4.0,4.0
OTHER,-106.0,106.0
WHITE,-289.0,289.0


PERCENT Difference between Actual and Expected Frequencies (Accident DF) (Actual - Expected) == 
 (Negative => Less than expected


Citation/Warning,Citation,Warning
Race,,
ASIAN,-3.8747,27.0090
BLACK,-0.1740,1.6938
HISPANIC,3.4882,-55.8938
NATIVE AMERICAN,-3.5604,25.4314
OTHER,-4.4441,29.6813
WHITE,-1.6797,14.0799


(Citation/Warning  Citation  Warning
 Race                               
 ASIAN                 2607      374
 BLACK                13298     1366
 HISPANIC             14982      935
 NATIVE AMERICAN        100       14
 OTHER                 2391      358
 WHITE                17184     2050,
 Citation/Warning      Citation      Warning
 Race                                       
 ASIAN              2708.013475   272.986525
 BLACK             13321.137067  1342.862933
 HISPANIC          14459.392982  1457.607018
 NATIVE AMERICAN     103.560395    10.439605
 OTHER              2497.258988   251.741012
 WHITE             17472.637094  1761.362906)

In [1]:
2607/374

6.970588235294118

In [2]:
14982/932

16.07510729613734

In [3]:
17184/2050

8.382439024390244

## Fatalities

Note that there have been few fatalities (434), so it's hard to draw any conclusions from this table.

In [11]:
chi_square_contingency_test(fatality_df)

Test statistic == 20.240673991875365
P-Value == 0.0270567605498549523801443683623801916838
Degrees of Freedom == 10
Contingency table == 


Violation Type,Citation,ESERO,Warning
Race,,,
ASIAN,21,0,4
BLACK,97,3,37
HISPANIC,62,2,32
NATIVE AMERICAN,0,0,1
OTHER,8,0,12
WHITE,86,3,66


Expected Frequencies == 


Violation Type,Citation,ESERO,Warning
Race,,,
ASIAN,16.0,0.0,9.0
BLACK,86.0,3.0,48.0
HISPANIC,61.0,2.0,34.0
NATIVE AMERICAN,1.0,0.0,0.0
OTHER,13.0,0.0,7.0
WHITE,98.0,3.0,54.0


Difference between Actual and Expected Frequencies (Fatality DF) (Actual - Expected) == 
 (Negative => 


Violation Type,Citation,ESERO,Warning
Race,,,
ASIAN,5.0,0.0,-5.0
BLACK,11.0,0.0,-11.0
HISPANIC,1.0,0.0,-2.0
NATIVE AMERICAN,-1.0,0.0,1.0
OTHER,-5.0,0.0,5.0
WHITE,-12.0,0.0,12.0


## Personal Injury

In [12]:
chi_square_contingency_test(personal_injury_df)

Test statistic == 67.71710453179797
P-Value == 0.0000000001220863624762970545741272547513
Degrees of Freedom == 10
Contingency table == 


Violation Type,Citation,ESERO,Warning
Race,,,
ASIAN,1060,1,111
BLACK,5004,9,356
HISPANIC,5333,5,309
NATIVE AMERICAN,41,0,1
OTHER,917,0,111
WHITE,6888,5,586


Expected Frequencies == 


Violation Type,Citation,ESERO,Warning
Race,,,
ASIAN,1088.0,1.0,83.0
BLACK,4982.0,5.0,382.0
HISPANIC,5240.0,5.0,401.0
NATIVE AMERICAN,39.0,0.0,3.0
OTHER,954.0,1.0,73.0
WHITE,6940.0,7.0,532.0


Difference between Actual and Expected Frequencies (Personal Injury DF) (Actual - Expected) == 
 (Negative => 


Violation Type,Citation,ESERO,Warning
Race,,,
ASIAN,-28.0,0.0,28.0
BLACK,22.0,4.0,-26.0
HISPANIC,93.0,0.0,-92.0
NATIVE AMERICAN,2.0,0.0,-2.0
OTHER,-37.0,-1.0,38.0
WHITE,-52.0,-2.0,54.0


# Plot Expected vs. Actual Frequencies

In [46]:
def plot_freqs(dataframe:pd.DataFrame):
    
    actual_df, expected_df = chi_square_contingency_test(dataframe=dataframe,
                                                         display=False)
    
    expected_df['value type'] = 'Expected'
    actual_df['value type'] = 'Actual'
    
    combined_df = pd.concat([expected_df, actual_df])
    
    combined_df = combined.melt(id_vars=['value type'], 
                                value_vars=['Citation', 'ESERO', 'SERO', 'Warning'],
                                ignore_index=False)
    
    # initialize figure
    fig = make_subplots(rows=2,
                        cols=3,
                        subplot_titles=actual_df.index,
                        horizontal_spacing = 0.05, 
                        vertical_spacing=.2, shared_yaxes=True)#, shared_yaxes=True)

    # create subplots
    for i, race in enumerate(actual_df.index):
        # get subplot position
        race_row = int(1 + np.floor(i/3))
        race_col = 1 + i%3
    
    
        # create subplot (NEW)
        fig.add_trace(px.bar(combined_df.loc[race], 
                             x="Violation Type",
                             y="value", 
                             color="value type",
                             barmode="group")
                      )
        
        # create subplot (OG)
        
        # fig.add_trace(go.Bar(x=actual_df.columns,
        #                     y=actual_df.loc[race],
        #                     opacity=.65,
        #                     name=race, 
        #                     yhoverformat=',.0%'), # percentage format for hovering
        #             row=race_row,
        #             col=race_col)
        
        # add a horizontal line for the population proportion
        # fig.add_shape(go.layout.Shape(type="line", 
        #                             yref="paper",
        #                             xref="x",
        #                             x0=-.5,
        #                             x1=len(crosstabs.columns),
        #                             y0=crosstabs.loc[race, "MC Pop."],
        #                             y1=crosstabs.loc[race, "MC Pop."],
        #                             line=dict(color='red', width=.75, dash='dash')
        #                             ),
        #               row=race_row,
        #               col=race_col)

    # manually configure size of figure
    # fig.update_layout(autosize=False,
    #                 width=1200,
    #                 height=820,
    #                 showlegend=False, title='Traffic Stop Statistics by Race',
    #                 title_x=0.5)

    # update y_axes to be identical across rows and to have percentage format
    # fig.update_yaxes(range=[0, .65],
    #                 tickformat=',.0%')

    # add explanation at the bottom
    # fig.update_layout(margin=dict(b=250))
    # fig.add_annotation(dict(font=dict(color='Black',size=15),
    #                                         x=0,
    #                                         y=-0.45,
    #                                         showarrow=False,
    #                                         text=explanation_text,
    #                                         textangle=0,
    #                                         xanchor='left',
    #                                         xref="paper",
    #                                         yref="paper"))

    fig.show()

    # save
    # fig.write_html("Traffic Stop Statistics by Race - Interactive.html")
    # fig.write_image("Traffic Stop Statistics by Race.png")

In [47]:
plot_freqs(accident_df)

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'value type=Expected<br>Violation Type=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Expected',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'Expected',
              'offsetgroup': 'Expected',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Citation', 'ESERO', 'SERO', 'Warning'], dtype=object),
              'xaxis': 'x',
              'y': array([2.71793015e+03, 1.35461097e+01, 5.37544035e-01, 2.73986195e+02]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'value type=Actual<br>Violation Type=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Actual',


In [40]:
combined

Violation Type,Citation,ESERO,SERO,Warning,value type
Race,,,,,
ASIAN,2717.930151,13.546110,0.537544,273.986195,Expected
BLACK,13336.483611,66.468768,2.637650,1344.409971,Expected
HISPANIC,14436.856530,71.953005,2.855278,1455.335187,Expected
NATIVE AMERICAN,104.883532,0.522737,0.020744,10.572987,Expected
OTHER,2500.929740,12.464584,0.494626,252.111050,Expected
WHITE,17464.916436,87.044795,3.454159,1760.584610,Expected
ASIAN,2607.000000,25.000000,0.000000,374.000000,Actual
BLACK,13298.000000,82.000000,4.000000,1366.000000,Actual
HISPANIC,14982.000000,47.000000,3.000000,935.000000,Actual


In [42]:
combined.melt(id_vars=['value type'], 
              value_vars=['Citation', 'ESERO', 'SERO', 'Warning'],
              ignore_index=False)

,value type,Violation Type,value
Race,,,
ASIAN,Expected,Citation,2717.930151
BLACK,Expected,Citation,13336.483611
HISPANIC,Expected,Citation,14436.856530
NATIVE AMERICAN,Expected,Citation,104.883532
OTHER,Expected,Citation,2500.929740
WHITE,Expected,Citation,17464.916436
ASIAN,Actual,Citation,2607.000000
BLACK,Actual,Citation,13298.000000
HISPANIC,Actual,Citation,14982.000000
